In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from sklearn.model_selection import train_test_split
import xgboost as xgb

from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score, f1_score
import pickle


In [5]:
pip install hyperopt

zsh:1: command not found: pip


In [6]:
pip install setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.4/863.4 kB 18.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pwd

'/Users/priyankachetry/Documents/Hands-on Projects/Credit_Card_Defaulter/codes'

In [4]:
df = pd.read_csv('../data/application_data.csv')
df = df.drop('SK_ID_CURR', axis=1)


In [5]:
# Default Rate or Event Rate
df.TARGET.mean()

0.08072881945686496

In [6]:
df.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Identify Continuous v/s Categorical Columns
categorical_var_list = list()
continuous_var_list = list()

for col_ in df.columns:
    if col_ != 'TARGET':
        if str(df[col_].dtype) == 'object':
            categorical_var_list.append(col_)
        else:
            continuous_var_list.append(col_)


In [8]:
df[categorical_var_list].head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,NaN,NaN,NaN,NaN
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,NaN,NaN,NaN


In [9]:
df[continuous_var_list].head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,-2437,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Calculate Missing Rates for all columns
df_missing_rate = (df.isnull().sum()*100/len(df)).reset_index()
df_missing_rate = df_missing_rate.rename(columns={'index': 'variable_name', 0: 'missing_rate'})
df_missing_rate


,variable_name,missing_rate
0,TARGET,0.000000
1,NAME_CONTRACT_TYPE,0.000000
2,CODE_GENDER,0.000000
3,FLAG_OWN_CAR,0.000000
4,FLAG_OWN_REALTY,0.000000
...,...,...
116,AMT_REQ_CREDIT_BUREAU_DAY,13.501631
117,AMT_REQ_CREDIT_BUREAU_WEEK,13.501631
118,AMT_REQ_CREDIT_BUREAU_MON,13.501631
119,AMT_REQ_CREDIT_BUREAU_QRT,13.501631


In [11]:
for col_ in continuous_var_list:
    df[col_] = df[col_].fillna(-99999)

for col_ in categorical_var_list:
    df[col_] = df[col_].fillna('-99999')


In [12]:
df.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
4,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Check correlation with Target

print(df[continuous_var_list+['TARGET']].corr()['TARGET'])

CNT_CHILDREN                  0.019187
AMT_INCOME_TOTAL             -0.003982
AMT_CREDIT                   -0.030369
AMT_ANNUITY                  -0.012696
AMT_GOODS_PRICE              -0.039545
                                ...   
AMT_REQ_CREDIT_BUREAU_WEEK   -0.032842
AMT_REQ_CREDIT_BUREAU_MON    -0.032843
AMT_REQ_CREDIT_BUREAU_QRT    -0.032843
AMT_REQ_CREDIT_BUREAU_YEAR   -0.032842
TARGET                        1.000000
Name: TARGET, Length: 105, dtype: float64


In [13]:
# Check correlation with Income

print(df[continuous_var_list+['TARGET']].corr()['AMT_INCOME_TOTAL'].abs().sort_values())

FLAG_PHONE                     0.000159
FLAG_DOCUMENT_20               0.000242
FLAG_DOCUMENT_10               0.000290
FLAG_MOBIL                     0.000325
FLAG_DOCUMENT_4                0.000529
                                 ...   
REGION_RATING_CLIENT_W_CITY    0.091735
AMT_CREDIT                     0.156870
AMT_GOODS_PRICE                0.159642
AMT_ANNUITY                    0.191345
AMT_INCOME_TOTAL               1.000000
Name: AMT_INCOME_TOTAL, Length: 105, dtype: float64


In [14]:
# Check # of unique categories/labels present in each categorical column

for col_ in categorical_var_list:
    print({col_: df[col_].unique()})
    print({col_: df[col_].nunique()})
    print('================================')


{'NAME_CONTRACT_TYPE': array(['Cash loans', 'Revolving loans'], dtype=object)}
{'NAME_CONTRACT_TYPE': 2}
{'CODE_GENDER': array(['M', 'F', 'XNA'], dtype=object)}
{'CODE_GENDER': 3}
{'FLAG_OWN_CAR': array(['N', 'Y'], dtype=object)}
{'FLAG_OWN_CAR': 2}
{'FLAG_OWN_REALTY': array(['Y', 'N'], dtype=object)}
{'FLAG_OWN_REALTY': 2}
{'NAME_TYPE_SUITE': array(['Unaccompanied', 'Family', 'Spouse, partner', 'Children',
       'Other_A', '-99999', 'Other_B', 'Group of people'], dtype=object)}
{'NAME_TYPE_SUITE': 8}
{'NAME_INCOME_TYPE': array(['Working', 'State servant', 'Commercial associate', 'Pensioner',
       'Unemployed', 'Student', 'Businessman', 'Maternity leave'],
      dtype=object)}
{'NAME_INCOME_TYPE': 8}
{'NAME_EDUCATION_TYPE': array(['Secondary / secondary special', 'Higher education',
       'Incomplete higher', 'Lower secondary', 'Academic degree'],
      dtype=object)}
{'NAME_EDUCATION_TYPE': 5}
{'NAME_FAMILY_STATUS': array(['Single / not married', 'Married', 'Civil marriage', 'Wido

In [15]:
df[categorical_var_list].head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,-99999,-99999,-99999,-99999
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,-99999,-99999,-99999,-99999
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,-99999,-99999,-99999,-99999


In [13]:
#stratifying ensures the distribution of y datasets are equal in training & testing sets
X = df.drop(columns='TARGET', axis=1) #Features
y = df.TARGET #Outcome/Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train.index = range(len(X_train))
X_test.index = range(len(X_test))
y_train.index = range(len(y_train))
y_test.index = range(len(y_test))

df_train = pd.concat([X_train, y_train], axis=1, ignore_index=False)
df_test = pd.concat([X_test, y_test], axis=1, ignore_index=False)


In [16]:
y_train.unique()

array([0, 1])

In [17]:
# Calculate WOE & IV
   ## First calculate WOE & IV for original data frame. then after spliting train, test dataset,
    ## calculate (replace df) woe, iv with reference train dataframe and finally populate calculated metrics of train data into test dataset.
    ## Train dataset is historical data & Test dataset is unseen data.


df_train["bads"] = 1-df_train["TARGET"]

total_good_events = df_train['TARGET'].sum()
total_bad_events = df_train['bads'].sum()

for col_ in categorical_var_list:
    print(col_)
    df_grouped = df_train.groupby(by=col_).agg({'TARGET': 'sum', 'bads': 'sum'}).reset_index()
    df_grouped['perc_goods'] = df_grouped.TARGET/total_good_events #calculated the percentage of good events
    df_grouped['perc_bads'] = df_grouped.bads/total_bad_events #calculated the percentage of bad events
    
    #WOE calculation
    df_grouped['woe']= np.log(df_grouped['perc_goods']/df_grouped['perc_bads'])
    
    #IV calculation - Just for info sake
    df_grouped['IV'] =  (df_grouped['perc_goods']- df_grouped['perc_bads'])*df_grouped['woe']
    # print(df_grouped.head())

    df_grouped.woe = df_grouped.woe.replace({-np.inf: 0, np.inf: 0})
    df_grouped.IV = df_grouped.IV.replace({-np.inf: 0, np.inf: 0})
    
    dict_woe = dict(zip(list(df_grouped[col_].values), list(df_grouped['woe'].values)))
    print(dict_woe)
    # print('==============================')
    df_train[col_] = df_train[col_].replace(dict_woe)
    df_test[col_] = df_test[col_].replace(dict_woe)


NAME_CONTRACT_TYPE
{'Cash loans': 0.03543028891413576, 'Revolving loans': -0.40203839417908405}
CODE_GENDER
{'F': -0.15477710005502712, 'M': 0.25113790661434543, 'XNA': 0.0}
FLAG_OWN_CAR
{'N': 0.05557880892351447, 'Y': -0.11596570504028508}


/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train[col_] = df_train[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test[col_] = df_test[col_].replace(dict_woe)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(u

FLAG_OWN_REALTY
{'N': 0.02401759705916845, 'Y': -0.010739810836794404}
NAME_TYPE_SUITE
{'-99999': -0.4232529350992987, 'Children': -0.08927696959916427, 'Family': -0.10322831358123223, 'Group of people': 0.06722960640978082, 'Other_A': 0.1136970056967624, 'Other_B': 0.228145828770791, 'Spouse, partner': -0.0277645856263129, 'Unaccompanied': 0.017699001397522902}
NAME_INCOME_TYPE


/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train[col_] = df_train[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test[col_] = df_test[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ve

{'Businessman': 0.0, 'Commercial associate': -0.06775718197727604, 'Maternity leave': 0.0, 'Pensioner': -0.431954325953546, 'State servant': -0.33869478502126316, 'Student': 0.0, 'Unemployed': 1.6440559332551203, 'Working': 0.18185869137057598}
NAME_EDUCATION_TYPE
{'Academic degree': -2.202215694610245, 'Higher education': -0.4328886273415675, 'Incomplete higher': 0.09749126263952984, 'Lower secondary': 0.34157981870076437, 'Secondary / secondary special': 0.10754340265672499}
NAME_FAMILY_STATUS


...

/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train[col_] = df_train[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test[col_] = df_test[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ve

FONDKAPREMONT_MODE
{'-99999': 0.06786068115360189, 'not specified': -0.03421843569670006, 'org spec account': -0.3041338520457744, 'reg oper account': -0.1538646307133372, 'reg oper spec account': -0.20742860040560207}
HOUSETYPE_MODE
{'-99999': 0.13339158393856615, 'block of flats': -0.1568067622219116, 'specific housing': 0.27623931989626654, 'terraced house': -0.047405814657570905}


/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test[col_] = df_test[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train[col_] = df_train[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ve

WALLSMATERIAL_MODE
{'-99999': 0.13003887073295872, 'Block': -0.13557422811546074, 'Mixed': -0.06206862169202822, 'Monolithic': -0.5540100408492907, 'Others': 0.09263241588164797, 'Panel': -0.2603857960304297, 'Stone, brick': -0.08666914828829303, 'Wooden': 0.23469312365091344}
EMERGENCYSTATE_MODE
{'-99999': 0.14650479381166526, 'No': -0.15425171264289764, 'Yes': 0.15134802892616772}


/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test[col_] = df_test[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train[col_] = df_train[col_].replace(dict_woe)
/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_35800/1405740799.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ve

In [18]:
df_train[categorical_var_list].head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,0.03543,-0.154777,-0.115966,-0.010740,0.017699,-0.067757,-0.432889,-0.074798,-0.037207,-0.608646,-0.039783,0.261621,-0.153865,-0.156807,-0.086669,-0.154252
1,0.03543,0.251138,0.055579,-0.010740,-0.103228,-0.338695,-0.432889,-0.074798,-0.037207,-0.292244,0.016383,0.304023,0.067861,0.133392,0.130039,0.146505
2,0.03543,0.251138,0.055579,-0.010740,0.017699,-0.067757,0.107543,-0.074798,-0.037207,0.297606,0.022199,0.147191,0.067861,0.133392,0.130039,0.146505
3,0.03543,-0.154777,0.055579,-0.010740,0.017699,-0.067757,0.107543,-0.074798,-0.037207,0.184144,0.028662,0.261621,0.067861,0.133392,0.130039,0.146505
4,0.03543,0.251138,-0.115966,0.024018,0.017699,-0.067757,0.107543,-0.074798,-0.037207,0.354446,0.022199,0.451149,-0.153865,-0.156807,-0.086669,-0.154252


In [19]:
X_train.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,F,Y,Y,2,405000.0,1971072.0,68643.0,1800000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cash loans,M,N,Y,0,337500.0,508495.5,38146.5,454500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,6.0
2,Cash loans,M,N,Y,1,112500.0,110146.5,13068.0,90000.0,Unaccompanied,...,0,0,0,0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
3,Cash loans,F,N,Y,2,40500.0,66384.0,3519.0,45000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
4,Cash loans,M,Y,N,0,225000.0,298512.0,31801.5,270000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
X_test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,F,N,Y,0,90000.0,803259.0,23616.0,670500.0,Family,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
1,Cash loans,F,N,Y,0,270000.0,1288350.0,41692.5,1125000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
2,Cash loans,F,N,Y,0,270000.0,253737.0,25227.0,229500.0,Unaccompanied,...,0,0,0,0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
3,Cash loans,F,N,Y,0,144000.0,436032.0,16564.5,360000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,2.0
4,Revolving loans,M,N,N,2,72000.0,225000.0,11250.0,225000.0,Unaccompanied,...,0,0,0,0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0


In [21]:
def dump_as_pickle(object=None, filename=None):
    with open(filename, 'wb') as file_:
        pickle.dump(object, file_)


def get_best_params(train_data, features, y, path_to_log_file):      ## Hyper parameter tuning: finding out the best parameter corresponding to 
                                                                       # the best model, which is chosen based on maximize(AUC) metric.
    space = {
        'max_depth': hp.choice('max_depth', [6]),
        'n_estimators': hp.choice('n_estimators', [60]),
        'reg_alpha': hp.choice('reg_alpha', [0, 0.01]),
        'reg_lambda': hp.choice('reg_lambda', [0.1, 1]),
        'min_child_weight': hp.choice('min_child_weight', [1, 5]),
        'colsample_bytree': hp.choice('colsample_bytree', [0.5, 1]),
        'scale_pos_weight': hp.choice('scale_pos_weight', [11])}

    def objective_xgboost(params):
        params = {
            'max_depth': int(params['max_depth']),
            'min_child_weight': int(params['min_child_weight']),
            'reg_alpha': params['reg_alpha'],
            'reg_lambda': params['reg_lambda'],
            'colsample_bytree': params['colsample_bytree'],
            'scale_pos_weight': int(params['scale_pos_weight']),
            'n_estimators': int(params['n_estimators'])
        }

        clf = XGBClassifier(
            learning_rate=0.03,
            n_jobs=os.cpu_count() - 1,
            **params
        )

        score = cross_val_score(clf, train_data[features], y, scoring='roc_auc', cv=4).mean()

        with open(path_to_log_file, 'a') as myfile:
            myfile.write('Params : {}\n'.format(params))
            myfile.write('AUC Score : {}\n'.format(score))
            myfile.write('===========================================\n')

        return {'loss': 1 - score, 'status': STATUS_OK}

    return fmin(fn=objective_xgboost, space=space, algo=tpe.suggest, max_evals=64)


def get_best_model(train_data=None, features=None, y=None, path_to_xgb_model=None, path_to_log_file=None):
    print('------------ Started Tuning XGB Model ------------')
    params_ = get_best_params(train_data, features, y, path_to_log_file)
    model_ = XGBClassifier(**params_)
    model_.fit(train_data[features], y)
    dump_as_pickle(object={'model': model_, 'features': features}, filename=path_to_xgb_model)


In [22]:
get_best_model(train_data=df_train, features=list(X_train.columns), y=y_train,
               path_to_xgb_model='../data/xgboost_model.pickle', path_to_log_file='../data/log.txt')


------------ Started Tuning XGB Model ------------
100%|██████████| 64/64 [09:26<00:00,  8.84s/trial, best loss: 0.2551399817332245]


In [23]:
df_train.to_csv('../data/df_train.csv', index=False)
df_test.to_csv('../data/df_test.csv', index=False)


# Load Model hyper parameters & Train, Test data

In [2]:
indep_vars = ['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAREA_MEDI',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'TOTALAREA_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR']


In [3]:
df_train = pd.read_csv('../data/df_train.csv')
df_test = pd.read_csv('../data/df_test.csv')


In [4]:
df_train.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET,bads
0,0.03543,-0.154777,-0.115966,-0.010740,2,405000.0,1971072.0,68643.0,1800000.0,0.017699,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,0.03543,0.251138,0.055579,-0.010740,0,337500.0,508495.5,38146.5,454500.0,-0.103228,...,0,0,0.0,0.0,0.0,0.0,0.0,6.0,0,1
2,0.03543,0.251138,0.055579,-0.010740,1,112500.0,110146.5,13068.0,90000.0,0.017699,...,0,0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,0,1
3,0.03543,-0.154777,0.055579,-0.010740,2,40500.0,66384.0,3519.0,45000.0,0.017699,...,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0,1
4,0.03543,0.251138,-0.115966,0.024018,0,225000.0,298512.0,31801.5,270000.0,0.017699,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [5]:
best_parms = {'max_depth': 6, 'min_child_weight': 1, 'reg_alpha': 0.01, 'reg_lambda': 0.1, 'colsample_bytree': 0.5, 'scale_pos_weight': 11, 'n_estimators': 60}
model_xgb = XGBClassifier(**best_parms)
model_xgb.fit(df_train[indep_vars], df_train.TARGET)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=60, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [18]:
# Calculate Feature Importance

feature_imp_dict = model_xgb.get_booster().get_score(importance_type='gain')
features_ = list(feature_imp_dict.keys())
values_ = list(feature_imp_dict.values())
df_feature_imp = pd.DataFrame({'feature': features_, 'importance': values_})
df_feature_imp = df_feature_imp.sort_values(by='importance', ascending=False)
df_feature_imp.index = range(len(df_feature_imp))
df_feature_imp['relative_importance'] = df_feature_imp.importance/df_feature_imp.importance.sum()
df_feature_imp.to_csv('../data/df_feature_importance_xgb.csv', index=False)


In [5]:
# Predict for Train data
df_train['predicted_default'] = model_xgb.predict(df_train[indep_vars])
pred_prob_list = list(model_xgb.predict_proba(df_train[indep_vars]))
pred_prob_list = [p[1] for p in pred_prob_list]
df_train['predicted_prob_of_default'] = pred_prob_list

# Predict for Test data
df_test['predicted_default'] = model_xgb.predict(df_test[indep_vars])
pred_prob_list = list(model_xgb.predict_proba(df_test[indep_vars]))
pred_prob_list = [p[1] for p in pred_prob_list]
df_test['predicted_prob_of_default'] = pred_prob_list


In [6]:
# Recall

recall_train = recall_score(df_train.TARGET, df_train.predicted_default)
recall_test = recall_score(df_test.TARGET, df_test.predicted_default)
print(recall_train, recall_test)


0.7612361167059907 0.6229860365198711


In [7]:
# Precision

precision_train = precision_score(df_train.TARGET, df_train.predicted_default)
precision_test = precision_score(df_test.TARGET, df_test.predicted_default)
print(precision_train, precision_test)


0.20979176248552805 0.17362670258943272


In [9]:
# F1 Score

f1_score_train = f1_score(df_train.TARGET, df_train.predicted_default)
f1_score_test = f1_score(df_test.TARGET, df_test.predicted_default)
print(f1_score_train, f1_score_test)

0.32893199055078953 0.2715673650942292


# Create buckets of probability scores and observe default rate (event rate), default capture rate across the buckets

In [10]:
# Train

df_train['pred_prob_bucket'] = pd.qcut(df_train['predicted_prob_of_default'], 10)
df_train['volume'] = 1              #volume here indicates no.of rows
df_train_grouped = df_train.groupby(by='pred_prob_bucket').agg(
    {'volume': 'sum', 'TARGET': 'sum', 'predicted_prob_of_default': 'mean'}).reset_index()
df_train_grouped['default_rate'] = df_train_grouped.TARGET/df_train_grouped.volume
df_train_grouped['perc_defaulters_captured'] = df_train_grouped.TARGET/df_train_grouped.TARGET.sum()
df_train_grouped = df_train_grouped.sort_values(by='predicted_prob_of_default', ascending=False)
df_train_grouped.index = range(len(df_train_grouped))

print(df_train_grouped)


  pred_prob_bucket  volume  TARGET  predicted_prob_of_default  default_rate  \
0    (0.701, 0.97]   21526    7004                   0.786303      0.325374   
1   (0.585, 0.701]   21526    3868                   0.640792      0.179690   
2   (0.494, 0.585]   21525    2488                   0.538167      0.115587   
3   (0.416, 0.494]   21526    1512                   0.453698      0.070241   
4   (0.347, 0.416]   21525    1137                   0.380760      0.052822   
5   (0.285, 0.347]   21526     679                   0.316088      0.031543   
6   (0.228, 0.285]   21526     390                   0.256458      0.018118   
7   (0.172, 0.228]   21525     222                   0.200354      0.010314   
8   (0.112, 0.172]   21526      64                   0.142886      0.002973   
9  (0.0005, 0.112]   21526      13                   0.072147      0.000604   

   perc_defaulters_captured  
0                  0.403062  
1                  0.222593  
2                  0.143178  
3         

/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_1146/2681097821.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train_grouped = df_train.groupby(by='pred_prob_bucket').agg(


In [11]:
# Test

df_test['pred_prob_bucket'] = pd.qcut(df_test['predicted_prob_of_default'], 10)
df_test['volume'] = 1
df_test_grouped = df_test.groupby(by='pred_prob_bucket').agg(
    {'volume': 'sum', 'TARGET': 'sum', 'predicted_prob_of_default': 'mean'}).reset_index()
df_test_grouped['default_rate'] = df_test_grouped.TARGET/df_test_grouped.volume
df_test_grouped['perc_defaulters_captured'] = df_test_grouped.TARGET/df_test_grouped.TARGET.sum()
df_test_grouped = df_test_grouped.sort_values(by='predicted_prob_of_default', ascending=False)
df_test_grouped.index = range(len(df_test_grouped))

print(df_test_grouped)


   pred_prob_bucket  volume  TARGET  predicted_prob_of_default  default_rate  \
0      (0.7, 0.976]    9226    2414                   0.785367      0.261652   
1      (0.584, 0.7]    9225    1400                   0.638958      0.151762   
2    (0.491, 0.584]    9225     897                   0.535987      0.097236   
3    (0.415, 0.491]    9226     768                   0.452586      0.083243   
4    (0.348, 0.415]    9225     555                   0.381108      0.060163   
5    (0.287, 0.348]    9225     450                   0.317184      0.048780   
6     (0.23, 0.287]    9226     363                   0.258358      0.039345   
7     (0.173, 0.23]    9225     269                   0.201299      0.029160   
8    (0.113, 0.173]    9225     200                   0.143819      0.021680   
9  (0.00084, 0.113]    9226     132                   0.072583      0.014307   

   perc_defaulters_captured  
0                  0.324114  
1                  0.187970  
2                  0.120435  

/var/folders/q9/pltwnf513ns14qtthspkrphm0000gn/T/ipykernel_1146/3165859574.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_test_grouped = df_test.groupby(by='pred_prob_bucket').agg(
